In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt

# Load the datasets
walmart_df = pd.read_csv('Dataset/wmt_historical_data.csv')
stellantis_df = pd.read_csv('Dataset/stla_historical_data.csv')


def clean_dataframe_for_ddm(df):
    """ Cleans df and sets Date index for DDM """
    df_copy = df.copy() # Work on a copy
    df_copy['Close'] = df_copy['Close'].astype(str).str.replace(',', '', regex=False)
    df_copy['Close'] = pd.to_numeric(df_copy['Close'], errors='coerce')
    df_copy.dropna(subset=['Close'], inplace=True)
    df_copy['Date'] = pd.to_datetime(df_copy['Date'].str[:-6], errors='coerce')
    df_copy.dropna(subset=['Date'], inplace=True)
    df_copy.sort_values('Date', inplace=True)
    # Set index for DDM
    df_copy.set_index('Date', inplace=True)
    # Ensure 'Dividends' column exists and is numeric
    if 'Dividends' in df_copy.columns:
         df_copy['Dividends'] = pd.to_numeric(df_copy['Dividends'], errors='coerce').fillna(0)
    else:
         print("Warning: Dividends column missing")
         df_copy['Dividends'] = 0
    return df_copy

# Re-load original dfs or ensure they are still available
# Assuming walmart_df and stellantis_df still hold the original loaded data:
walmart_df_ddm = clean_dataframe_for_ddm(walmart_df)
stellantis_df_ddm = clean_dataframe_for_ddm(stellantis_df)

print("\nDataFrames prepared with DatetimeIndex for DDM calculations.")

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (



DataFrames prepared with DatetimeIndex for DDM calculations.


## Walmart and Stellantis dividends

In [2]:
# Store the dividends in a new DataFrame and filter out non-positive dividends
walmart_dividends = walmart_df_ddm.loc[walmart_df_ddm['Dividends'] > 0, 'Dividends']

# Display the dividends for Walmart
print(f"--- Walmart Dividends ---")
for date, dividend in walmart_dividends.items():
    date_only = str(date).split(' ')[0]  # Extract only the date part
    print(f"Walmart distributed {dividend} dividend on {date_only}")


# Store the dividends in a new DataFrame and filter out non-positive dividends
stellantis_dividends = stellantis_df_ddm.loc[stellantis_df_ddm['Dividends'] > 0, 'Dividends']

# Display the dividends for Stellantis
print(f"--- Stellantis Dividends ---")
for date, dividend in stellantis_dividends.items():
    date_only = str(date).split(' ')[0]  # Extract only the date part
    print(f"Stellantis distributed {dividend} dividend on {date_only}")


--- Walmart Dividends ---
Walmart distributed 0.18 dividend on 2020-05-01
Walmart distributed 0.18 dividend on 2020-08-01
Walmart distributed 0.18 dividend on 2020-12-01
Walmart distributed 0.183333 dividend on 2021-03-01
Walmart distributed 0.183333 dividend on 2021-05-01
Walmart distributed 0.183333 dividend on 2021-08-01
Walmart distributed 0.183333 dividend on 2021-12-01
Walmart distributed 0.186667 dividend on 2022-03-01
Walmart distributed 0.186667 dividend on 2022-05-01
Walmart distributed 0.186667 dividend on 2022-08-01
Walmart distributed 0.186667 dividend on 2022-12-01
Walmart distributed 0.19 dividend on 2023-03-01
Walmart distributed 0.19 dividend on 2023-05-01
Walmart distributed 0.19 dividend on 2023-08-01
Walmart distributed 0.19 dividend on 2023-12-01
Walmart distributed 0.208 dividend on 2024-03-01
Walmart distributed 0.208 dividend on 2024-05-01
Walmart distributed 0.208 dividend on 2024-08-01
Walmart distributed 0.208 dividend on 2024-12-01
Walmart distributed 0.235 

## Average growth rate

In [3]:
# Annualize the dividends to compute the annual rate of growth
walmart_dividends = walmart_dividends.drop_duplicates()
walmart_dividends = walmart_dividends * 4

# Compute percentage change
walmart_dividends_pct_change = walmart_dividends.pct_change() * 100

# Store the results in a DataFrame for better visualization
walmart_dividends_df = pd.DataFrame({
    "Dividends": walmart_dividends,
    "pct_change": walmart_dividends_pct_change
})

# Compute average growth rate
avg_growth_rate = walmart_dividends_pct_change.mean()
print(f"--- Walmart Average Dividend Growth Rate ---")
print(f"Average Growth Rate: {avg_growth_rate:.2f}%")


--- Walmart Average Dividend Growth Rate ---
Average Growth Rate: 5.58%


In [4]:
# Sum the dividens that were distributed in the same year
stellantis_dividends_df = pd.DataFrame(stellantis_dividends)
stellantis_dividends_df['Year'] = stellantis_dividends_df.index.year
stellantis_dividends_df = stellantis_dividends_df.groupby('Year').sum()

# Compute percentage change
stellantis_dividends_pct_change = stellantis_dividends_df['Dividends'].pct_change() * 100

# Store the results in a DataFrame for better visualization
stellantis_dividends_df = pd.DataFrame({
    "Dividends": stellantis_dividends_df['Dividends'],
    "pct_change": stellantis_dividends_pct_change
})

# Compute average growth rate
avg_growth_rate_stellantis = stellantis_dividends_pct_change.mean()
print(f"--- Stellantis Average Dividend Growth Rate ---")
print(f"Average Growth Rate: {avg_growth_rate_stellantis:.2f}%")


--- Stellantis Average Dividend Growth Rate ---
Average Growth Rate: -5.19%


In [5]:
# 1. Calculate D0 (Last Annual Dividend per Share)
try:
    # ... (Keep the D0 calculation logic as before) ...
    last_date_wmt = walmart_df_ddm.index.max()
    start_date_wmt = last_date_wmt - pd.DateOffset(months=11)
    wmt_dividends_last_year = walmart_df_ddm.loc[start_date_wmt:last_date_wmt, 'Dividends'].sum()
    wmt_last_quarterly_dividend = walmart_df_ddm['Dividends'].iloc[-1] if walmart_df_ddm['Dividends'].iloc[-1] > 0 else 0.235
    wmt_D0_alternative = wmt_last_quarterly_dividend * 4
    # print(f"(Info: D0 based on last 12m sum: {wmt_dividends_last_year:.3f}, D0 based on quarterly*4: {wmt_D0_alternative:.3f})")
    wmt_D0 = wmt_D0_alternative # Using quarterly based
except Exception as e:
    print(f"Error calculating Walmart D0: {e}")
    wmt_D0 = 0

# 2. Cost of Equity (Re) - Use the CAPM calculated value
try:
    Re_ddm_wmt = pd.read_csv('Outputs\expected_returns.csv').iloc[0, 1] # Assuming the CAPM value is stored in a CSV
    print(f"CAPM Cost of Equity (Re) for Walmart: {Re_ddm_wmt}")
except NameError:
    print("Warning: Re_walmart not found from CAPM calculation. Using placeholder 0.0711")
    Re_ddm_wmt = 0.0711

# 3. Dividend Growth Rate (g) - Placeholder Assumption
g_ddm_wmt = 0.06 # Approximate: Assume 3.5% constant dividend growth

print(f"\n--- DDM Calculation for Walmart (Approximate - Gordon Growth) ---")
print(f"Using Annual Dividend (D0): ${wmt_D0:.3f}")
print(f"Cost of Equity (Re):        {Re_ddm_wmt:.2%}")
print(f"Assumed Constant Div Growth (g): {g_ddm_wmt:.1%}")
print("-" * 40)

walmart_share_price = 98.61

# --- Perform DDM Calculation ---
intrinsic_value_ps_ddm_wmt = float('nan') # Initialize
if wmt_D0 <= 0:
    print("Cannot calculate DDM because D0 is zero or negative.")
elif g_ddm_wmt >= Re_ddm_wmt:
    print("Error: Dividend growth rate (g) cannot be >= Cost of Equity (Re) for Gordon Growth Model.")
else:
    wmt_D1 = wmt_D0 * (1 + g_ddm_wmt)
    print(f"Expected Next Dividend (D1): ${wmt_D1:.3f}")
    intrinsic_value_ps_ddm_wmt = wmt_D1 / (Re_ddm_wmt - g_ddm_wmt)
    print(f"Intrinsic Value per Share:   ${intrinsic_value_ps_ddm_wmt:.2f}")
    print(f"Current Market Price:        ${walmart_share_price:.2f}")

    if intrinsic_value_ps_ddm_wmt > walmart_share_price:
        print("--> DDM suggests stock may be UNDERVALUED (using approximate assumptions)")
    elif intrinsic_value_ps_ddm_wmt < walmart_share_price:
         print("--> DDM suggests stock may be OVERVALUED (using approximate assumptions)")
    else:
         print("--> DDM suggests stock may be FAIRLY VALUED (using approximate assumptions)")

# --- Commentary on Walmart DDM Result ---
print("\nDDM Commentary for Walmart:")
if not np.isnan(intrinsic_value_ps_ddm_wmt) and intrinsic_value_ps_ddm_wmt < walmart_share_price:
    print(f"The calculated DDM value (${intrinsic_value_ps_ddm_wmt:.2f}) is significantly lower than the market price (${walmart_share_price:.2f}).")
    print("This is primarily driven by the low assumed constant growth rate (g = {g_ddm_wmt:.1%}).")
    # Sensitivity Example
    g_test = 0.04
    if g_test < Re_ddm_wmt:
        val_test = (wmt_D0 * (1 + g_test)) / (Re_ddm_wmt - g_test)
        print(f"A higher, potentially more realistic long-term growth rate (e.g., g = {g_test:.1%}) would yield a value of ${val_test:.2f}, still below market.")
    print("This suggests the simple Gordon Growth DDM may not fully capture market expectations or other value drivers for Walmart, or the market applies a lower discount rate/higher growth.")
    print("Consider comparing with DCF and Comparables, or using a multi-stage DDM.")

FileNotFoundError: [Errno 2] No such file or directory: 'Outputs\\expected_returns.csv'

In [ ]:
# 1. Calculate D0 (Last Annual Dividend per Share in EUR)
stla_D0_eur = 1.652
print(f"(Info: Using manually set D0 = {stla_D0_eur:.3f} EUR based on last known annual payment)")

try:
    Re_ddm_stla = Re_stellantis
except NameError:
    print("Warning: Re_stellantis not found from CAPM calculation. Using placeholder 0.1078")
    Re_ddm_stla = 0.1078 # Placeholder if needed

# 3. Dividend Growth Rate (g) - Placeholder Assumption (ADJUST BASED ON RESEARCH)
g_ddm_stla = 0.020 # Approximate: Assume 2.0% constant dividend growth

# 4. Exchange Rate (Use the same as before - ADJUST)
try:
    eur_to_usd_rate # Check if it exists from Cell 140
except NameError:
    eur_to_usd_rate = 1.08 # Define if not already set
    print(f"Using default EUR/USD rate: {eur_to_usd_rate}")


print(f"\n--- DDM Calculation for Stellantis (Approximate) ---")
print(f"Using Annual Dividend (D0, EUR): €{stla_D0_eur:.3f}")
print(f"Cost of Equity (Re):            {Re_ddm_stla:.2%}")
print(f"Assumed Div Growth (g):         {g_ddm_stla:.1%}")
print(f"Assumed EUR/USD Rate:           {eur_to_usd_rate:.4f}")
print("-" * 40)

# --- Perform DDM Calculation (in USD) ---
if stla_D0_eur <= 0: # Check if D0 is valid
     print("Cannot calculate DDM because D0 is zero or negative.")
elif g_ddm_stla >= Re_ddm_stla:
    print("Error: Dividend growth rate (g) cannot be >= Cost of Equity (Re) for Gordon Growth Model.")
    intrinsic_value_ps_ddm_stla = float('nan')
else:
    # Calculate D1 in EUR
    stla_D1_eur = stla_D0_eur * (1 + g_ddm_stla)
    # Convert D1 to USD
    stla_D1_usd = stla_D1_eur * eur_to_usd_rate
    print(f"Expected Next Dividend (D1, EUR): €{stla_D1_eur:.3f}")
    print(f"Expected Next Dividend (D1, USD): ${stla_D1_usd:.3f}")

    # Calculate Intrinsic Value in USD
    intrinsic_value_ps_ddm_stla = stla_D1_usd / (Re_ddm_stla - g_ddm_stla)

    print(f"Intrinsic Value per Share (USD): ${intrinsic_value_ps_ddm_stla:.2f}")
    print(f"Current Market Price (USD):      ${s_share_price:.2f}") # s_share_price from Cell 135

    if intrinsic_value_ps_ddm_stla > s_share_price:
        print("--> DDM suggests stock may be UNDERVALUED (using approximate assumptions)")
    elif intrinsic_value_ps_ddm_stla < s_share_price:
         print("--> DDM suggests stock may be OVERVALUED (using approximate assumptions)")
    else:
         print("--> DDM suggests stock may be FAIRLY VALUED (using approximate assumptions)")

